In [1]:
import os
import warnings
import datacube
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt


import sys
sys.path.insert(1, '/home/jovyan/dev/Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.dask import create_local_dask_cluster
from dea_tools.plotting import rgb

warnings.filterwarnings('ignore')

In [2]:
client = create_local_dask_cluster(return_client = True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,Workers: 1
Total threads: 3,Total memory: 29.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42995,Workers: 1
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/8787/status,Total threads: 3
Started: Just now,Total memory: 29.45 GiB
Comm: tcp://127.0.0.1:42949,Total threads: 3
Dashboard: /user/jenna.guffogg@ga.gov.au/proxy/36103/status,Memory: 29.45 GiB
Nanny: tcp://127.0.0.1:42375,


In [9]:
# region_code = ['x176y085'] #marysville
# region_code = ['x168y092'] #hopetoun
# region_code = ['x140y138'] #west macdonnell
# region_codes = ['x148y166'] #limmen NT - cloud heavy area
# region_codes = ['x175y066'] # Tas SW nat park


region_codes = ['x176y085']

time='2022'
resolution=(-30,30)
filters_id = 'test'
measurements=['nbart_green', 'nbart_red', 'nbart_blue']
dask_chunks = dict(x=1000, y=1000)

In [10]:
dc = datacube.Datacube(app='fc_ls_test')

query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'dask_chunks': dask_chunks,
    'group_by': 'solar_day',
    'output_crs': 'EPSG:3577'
}

In [11]:
# open tiles and select

gdf = gpd.read_file('~/gdata1/projects/fc-sub-annual/data/testing_minitile_suite.geojson')

gdf = gdf[gdf['region_code'].isin(region_codes)]

In [14]:
# gdf.explore(
#     tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#     attr='Esri',
#     name='Esri satellite'
# )

In [ ]:
def fc_run_function(dc, geom, query, time, region_code):
    # update the query to include our geometry
    query.update({'geopolygon': geom})

    # load available data from landsat
    ls_fc = load_ard(dc=dc,
                    products=['ga_ls9c_ard_3', 'ga_ls8c_ard_3'],
                     cloud_mask='fmask',
                     mask_pixel_quality=True,
                     mask_contiguity=True,
                     skip_broken_datasets=True,
                     verbose=False,
                    **query)
    #-----fractional-cover-----#

    #-----export-tiles-----#
    tiles_folder = f'/gdata/projects/fc-sub-annual/results/{filters_id}'
    if not os.path.exists(tiles_folder):
        os.makedirs(tiles_folder)